Tree, Binary search trees

In [ ]:
# Submitted by: CHOO YU LIANG (U1840932K)
class TreeNode:
    def __init__(self, initKey, initParent=None, initLeftChild=None, initRightChild=None):
        self.key = initKey
        self.parent = initParent
        self.leftChild = initLeftChild
        self.rightChild = initRightChild
        self.nw = 1     # Stores rank of node


class BinarySearchTree:
    def __init__(self, initRootNode=None):
        self.root = initRootNode
        self.treesize = 0
        if initRootNode is not None:
            self.treesize += 1

    # IMPLEMENT HERE
    # return the node with key value = k, otherwise return None
    def search(self, k, currNode):
        if self.root is not None:
            if k == currNode.key:
                return currNode
            elif k < currNode.key and currNode.leftChild is not None:
                return self.search(k, currNode.leftChild)
            elif k > currNode.key and currNode.rightChild is not None:
                return self.search(k, currNode.rightChild)
            else:
                return None
        else:
            return "Error: Tree is empty"
       
    # IMPLEMENT HERE
    # create a new node with key value = k, and insert it into the tree, and return the newly created node
    # Note: nw of all affected nodes needs to be updated here
    def insert(self, k, currNode):
        # Search for location to insert node
        if self.root != None:   # If tree is non-empty
            if k < currNode.key:
                if currNode.leftChild is None:
                    currNode.leftChild = TreeNode(k, currNode)
                    # Increment nw count
                    currNode.nw += 1
                    self.treesize += 1
                    return currNode.leftChild
                else:
                    # We visited this node when looking to insert
                    currNode.nw += 1
                    self.insert(k, currNode.leftChild)
            elif k > currNode.key:
                if currNode.rightChild is None:
                    currNode.rightChild = TreeNode(k, currNode)
                    # Increment nw count
                    currNode.nw += 1
                    self.treesize += 1
                    return currNode.rightChild
                else:
                    # We visited this node when looking to insert
                    currNode.nw += 1
                    self.insert(k, currNode.rightChild)
            else:
                return "Error: Value exists in tree"
        else:
            self.root = TreeNode(k)   # If tree is empty, insert as root
            self.treesize += 1
            return self.root

    # IMPLEMENT HERE
    # delete the node with key value = k; do nothing if not found
    # Note: nw of all affected nodes needs to be updated here
    def delete(self, k):
        if self.root is not None:
            # Search for node with key value = k
            node = self.search(k, currNode=self.root)

            # Function to count number of children of deletion node
            def num_child(node):
                count = 0
                if node.leftChild:
                    count += 1
                if node.rightChild:
                    count += 1
                return count

            # Function to find successor node for a deletion node with two children
            def find_successor(node):
                successor = node.rightChild
                while successor.leftChild:
                    successor = successor.leftChild
                else:
                    return successor

            # Function that updates all nw counts of affected nodes after deletion
            def update_nw(node):
                while node.parent:
                    node = node.parent
                    node.nw -= 1
                else:
                    return

            # If node containing k is found in the tree, consider the cases where
            # (1) node has 0 children, (2) node has 1 children and (3) node has 2 children
            if node is not None:
                # Get number of children of node
                child_cnt = num_child(node)

                # Consider case 1
                if child_cnt == 0:
                    if node == self.root:
                        self.root = None
                        self.treesize = 0
                    if node.parent.leftChild == node:
                        update_nw(node)
                        node.parent.leftChild = None
                        node.parent = None
                        self.treesize -= 1
                    if node.parent.rightChild == node:
                        update_nw(node)
                        node.parent.rightChild = None
                        node.parent = None
                        self.treesize -= 1
                    
                # Consider case 2
                elif child_cnt == 1:
                    # If node has children, move its children up one level
                    # by considering whether node is parent's left or right child
                    if node.leftChild:
                        # If node is root, its child is now root    
                        if node == self.root:
                            self.root = node.leftChild
                            (node.leftChild).parent = None
                            self.treesize -= 1
                        if (node.parent).leftChild == node:
                            (node.leftChild).parent = node.parent
                            (node.parent).leftChild = node.leftChild
                            update_nw(node.leftChild)
                            self.treesize -= 1
                        if (node.parent).rightChild == node:
                            (node.leftChild).parent = node.parent
                            update_nw(node.leftChild)
                            (node.parent).rightChild = node.leftChild
                            self.treesize -= 1
                    if node.rightChild:
                        if node == self.root:
                            self.root = node.rightChild
                            (node.rightChild).parent = None
                            self.treesize -= 1
                        if (node.parent).leftChild == node:
                            (node.rightChild).parent = node.parent
                            update_nw(node.rightChild)
                            (node.parent).leftChild = node.rightChild
                            self.treesize -= 1
                        if (node.parent).rightChild == node:
                            (node.rightChild).parent = node.parent
                            update_nw(node.rightChild)
                            (node.parent).rightChild = node.rightChild
                            self.treesize -= 1
      
                # Consider case 3
                else:
                    # Find in-order successor node of deletion node starting from its right subtree
                    successor = find_successor(node)
                    # Handle special exception where right subtree of deletion node has no left subtree
                    # In this case, successor is simply the right subtree of deletion node
                    if successor == node.rightChild:
                        # Copy successor's key value to deletion node
                        node.key = successor.key
                        # If successor has a right child, we replace successor with its right child
                        if successor.rightChild:
                            (successor.rightChild).parent = successor.parent
                            update_nw(successor.rightChild)
                            (successor.parent).rightChild = successor.rightChild
                        else:
                            # We update nw count then delete successor node
                            update_nw(successor)
                            successor.parent = None
                            node.rightChild = None
                        self.treesize -= 1
                    # General case where right subtree of deletion node has a left subtree
                    # so successor is the left-most internal node of right subtree of 
                    # deletion node. Note that in this case, successor will never have left child
                    else:
                        # Copy successor's key value to deletion node
                        node.key = successor.key
                        # If successor has a right child, we replace successor with its right child
                        if successor.rightChild:
                            if (successor.parent).leftChild == successor:
                                (successor.rightChild).parent = successor.parent
                                update_nw(successor.rightChild)
                                (successor.parent).leftChild = successor.rightChild
                        else:
                            # We update nw count then delete successor node
                            update_nw(successor)
                            (successor.parent).leftChild = None
                            successor.parent = None
                        self.treesize -= 1
            else:
                return "Error: Key does not exist"
        else:
            return "Error: Tree is empty"

    # IMPLEMENT HERE
    # search range [minimum, maximum] both inclusive; assume minimum <= maximum already guaranteed
    # return the list of key values in the range
    def searchByRange(self, minimum, maximum, currNode):
        if currNode == None:
            return []
        if currNode.key < minimum:
            return self.searchByRange(minimum, maximum, currNode.rightChild)
        elif currNode.key >= minimum and currNode.key <= maximum:
            return self.searchByRange(minimum, maximum, currNode.leftChild) + [currNode.key] + self.searchByRange(minimum, maximum, currNode.rightChild)
        else:
            return self.searchByRange(minimum, maximum, currNode.leftChild)

    # IMPLEMENT HERE
    # search and return the i-th smallest node, return None if i is out of range
    def searchith(self, i, currNode):
        # Check if index is out of bounds
        if i <= 0 or i > self.treesize:
            return None
        # Check if left subtree exists, otherwise it is an external node so we set nw to 0
        if currNode.leftChild is not None:
            ls_rank = currNode.leftChild.nw
        else:
            ls_rank = 0
        # Following the algorithm given 
        if i < ls_rank + 1:
            return self.searchith(i, currNode.leftChild)
        elif i > ls_rank + 1:
            return self.searchith(i-ls_rank-1, currNode.rightChild)
        elif i == ls_rank + 1:
            return currNode

    # print the tree layer by layer, in increasing order of depth       
    def printTree(self):
        x = [self.root]              
        print("--------- Tree begins here ----------")
        NonEmptyLevel = True
        while NonEmptyLevel == True:
            y = []
            NonEmptyLevel = False
            for i in range(len(x)):
                if x[i] != None:
                    print(x[i].key, end=' ')
                    NonEmptyLevel = True
                    y += [x[i].leftChild, x[i].rightChild]
                else:
                    print(' ', end=' ')
            x = y
            print()
        print("--------- Tree ends here -------------")

    def inorder(self, currNode):
        if currNode.leftChild:
            self.inorder(currNode.leftChild)
        print(currNode.key, currNode.nw)
        if currNode.rightChild:
            self.inorder(currNode.rightChild)
            
    def preorder(self, currNode):
        print(currNode.key, end='|')
        if currNode.leftChild:
            self.preorder(currNode.leftChild)
        if currNode.rightChild:
            self.preorder(currNode.rightChild)


In [ ]:
bst = BinarySearchTree(TreeNode(10))
bst.insert(5, bst.root)
bst.insert(4, bst.root)
bst.insert(7, bst.root)
bst.insert(6, bst.root)
bst.insert(8, bst.root)
bst.insert(11, bst.root)

bst.printTree()
print('\n')
bst.preorder(bst.root)

--------- Tree begins here ----------
10 
5 11 
4 7     
    6 8 
        
--------- Tree ends here -------------


10|5|4|7|6|8|11|

In [ ]:
bst = BinarySearchTree(TreeNode('H'))
bst.insert('D', bst.root)
bst.insert('J', bst.root)
bst.insert('K', bst.root)
bst.insert('C', bst.root)
bst.insert('F', bst.root)
bst.insert('M', bst.root)
bst.insert('B', bst.root)
bst.insert('A', bst.root)
bst.insert('P', bst.root)
bst.insert('L', bst.root)
bst.insert('T', bst.root)
bst.insert('Z', bst.root)
bst.printTree()

bst.delete('H')
bst.printTree()

print(bst.searchByRange('A', 'Z', bst.root))
print(bst.searchByRange('K', 'M', bst.root))

print(bst.searchith(8, bst.root).key)

bst.delete('K')
bst.printTree()
print(bst.searchith(8, bst.root).key)

print(bst.search('C', bst.root).key)
print(bst.search('N', bst.root))


# --------- Tree begins here ----------
# H 
# D J 
# C F   K 
# B         M 
# A   L P 
#           T 
#   Z 
    
# --------- Tree ends here -------------
# --------- Tree begins here ----------
# J 
# D K 
# C F   M 
# B       L P 
# A         T 
#       Z 
    
# --------- Tree ends here -------------
# ['A', 'B', 'C', 'D', 'F', 'J', 'K', 'L', 'M', 'P', 'T', 'Z']
# ['K', 'L', 'M']
# L
# --------- Tree begins here ----------
# J 
# D M 
# C F L P 
# B             T 
# A     Z 
        
# --------- Tree ends here -------------
# M
# C
# None

--------- Tree begins here ----------
H 
D J 
C F   K 
B         M 
A   L P 
          T 
  Z 
    
--------- Tree ends here -------------
--------- Tree begins here ----------
J 
D K 
C F   M 
B       L P 
A         T 
      Z 
    
--------- Tree ends here -------------
['A', 'B', 'C', 'D', 'F', 'J', 'K', 'L', 'M', 'P', 'T', 'Z']
['K', 'L', 'M']
L
--------- Tree begins here ----------
J 
D M 
C F L P 
B             T 
A     Z 
        
--------- Tree ends here -------------
M
C
None
